In [1]:
import db.knowhere_db as kdb
import pipeline.pipeline as pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
glen_H_data_raw = pd.read_csv("Labeled_glen_2017-03-19 to 2017-03-23.csv")


In [3]:
test_labeled = pd.read_csv("glen_3_24_labeled.csv", index_col = 0)

In [4]:
glen_H_data = glen_H_data_raw[['Acceleration x','Acceleration y','Acceleration z', 'Altimeter (Barometer) Pressure',\
                               'Microphone Left Channel Level', 'Microphone Right Channel Level','Magnetometer x',\
                               'Magnetometer y','Magnetometer z','Gyrometer x','Gyrometer y',\
                               'Gyrometer z', 'classification']]
glen_H_data = glen_H_data.dropna()
glen_H_data.head()

,Acceleration x,Acceleration y,Acceleration z,Altimeter (Barometer) Pressure,Microphone Left Channel Level,Microphone Right Channel Level,Magnetometer x,Magnetometer y,Magnetometer z,Gyrometer x,Gyrometer y,Gyrometer z,classification
9,0.035286,-0.032134,-0.174451,101.297173,-22.18490,-22.18490,-48.244934,139.584091,-442.558044,-0.189776,-0.747892,0.672498,D
10,0.045886,-0.052843,0.180941,101.301842,-5.64593,-5.64593,-48.244934,139.584091,-442.558044,0.186489,1.126667,0.098381,D
11,0.037488,-0.080072,0.024071,101.297859,-5.64593,-5.64593,-48.244934,139.584091,-442.558044,0.186489,1.126667,0.098381,D
12,0.104773,-0.102392,0.040786,101.294731,-5.64593,-5.64593,-48.244934,139.584091,-442.558044,0.186489,1.126667,0.098381,D
13,0.196326,-0.233689,-0.079535,101.297264,-5.64593,-5.64593,-48.244934,139.584091,-442.558044,0.186489,1.126667,0.098381,D


In [5]:
test = test_labeled[['Acceleration x','Acceleration y','Acceleration z', 'Altimeter (Barometer) Pressure',\
                               'Microphone Left Channel Level', 'Microphone Right Channel Level','Magnetometer x',\
                               'Magnetometer y','Magnetometer z','Gyrometer x','Gyrometer y',\
                               'Gyrometer z', 'classification']]
test.head()

,Acceleration x,Acceleration y,Acceleration z,Altimeter (Barometer) Pressure,Microphone Left Channel Level,Microphone Right Channel Level,Magnetometer x,Magnetometer y,Magnetometer z,Gyrometer x,Gyrometer y,Gyrometer z,classification
2017-03-24 06:31:10,0.000000,0.000000,0.000000,NaN,NaN,NaN,-65.585632,113.658890,-441.740631,0.195217,0.169663,-0.200276,W
2017-03-24 06:31:15,-0.023296,-0.018811,0.066937,102.441696,-3.34134,-3.34134,-66.959152,121.556625,-445.337341,0.057329,-0.160387,-0.636138,W
2017-03-24 06:31:19,-0.015955,0.008646,0.017675,102.441505,-3.34134,-3.34134,-66.959152,121.556625,-445.337341,0.057329,-0.160387,-0.636138,W
2017-03-24 06:31:24,-0.027630,-0.027705,-0.014916,102.440567,-16.32120,-16.32120,-62.323517,116.577621,-443.865967,0.077324,-0.622623,0.095855,W
2017-03-24 06:31:28,0.080779,-0.055970,-0.106723,102.443634,-1.21351,-1.21351,-103.872543,115.204086,-422.612701,0.255691,-0.515937,-1.319237,W


In [6]:
glen_H_data = glen_H_data.replace(to_replace='T_E', value='T_S')
glen_H_data = glen_H_data.replace(to_replace='E_U', value='E')
glen_H_data = glen_H_data.replace(to_replace='E_D', value='E')
glen_H_data = glen_H_data.replace(to_replace='S_U', value='W')
glen_H_data = glen_H_data.replace(to_replace='T_T', value='T_S')


glen_H_data.groupby("classification").size()

classification
D      1749
E       885
S       251
T_D    1159
T_S     789
W      1359
dtype: int64

In [7]:
test = test.replace(to_replace='T_E', value='T_S')
test = test.replace(to_replace='E_U', value='E')
test = test.replace(to_replace='E_D', value='E')
test = test.replace(to_replace='S_U', value='W')
test = test.replace(to_replace='T_T', value='T_S')


test.groupby("classification").size()

classification
D      150
E        8
S       37
T_D    297
T_S     84
W      255
dtype: int64

In [8]:
glen_H_data['Acceleration'] =  np.sqrt(glen_H_data['Acceleration x']**2 + glen_H_data['Acceleration y']**2 +\
                                       glen_H_data['Acceleration z']**2)
glen_H_data['Magnetometer'] =  np.sqrt(glen_H_data['Magnetometer x']**2 + glen_H_data['Magnetometer y']**2 +\
                                       glen_H_data['Magnetometer z']**2)
glen_H_data['Gyrometer']    =  np.sqrt(glen_H_data['Gyrometer x']**2 + glen_H_data['Gyrometer y']**2 +\
                                       glen_H_data['Gyrometer z']**2)
glen_H_data['Microphone']    =  (glen_H_data['Microphone Left Channel Level'] +\
                                glen_H_data['Microphone Right Channel Level'])/2
glen_H_data = glen_H_data[['Acceleration','Magnetometer','Gyrometer','Microphone','Altimeter (Barometer) Pressure', 'classification']]

### Prep Test Data

In [9]:
test['Acceleration'] =  np.sqrt(test['Acceleration x']**2 + test['Acceleration y']**2 +\
                                       test['Acceleration z']**2)
test['Magnetometer'] =  np.sqrt(test['Magnetometer x']**2 + test['Magnetometer y']**2 +\
                                       test['Magnetometer z']**2)
test['Gyrometer']    =  np.sqrt(test['Gyrometer x']**2 + test['Gyrometer y']**2 +\
                                       test['Gyrometer z']**2)
test['Microphone']    =  (test['Microphone Left Channel Level'] +\
                                test['Microphone Right Channel Level'])/2
test = test[['Acceleration','Magnetometer','Gyrometer','Microphone','Altimeter (Barometer) Pressure', 'classification']]

### set window


In [10]:
window = 50

In [11]:

# Rolling Means
glen_H_data['RollingMeanAcceleration'] = glen_H_data['Acceleration'].rolling(window=window,center=False).mean()
glen_H_data['RollingMeanMagnetometer'] = glen_H_data['Magnetometer'].rolling(window=window,center=False).mean()
glen_H_data['RollingMeanGyrometer'] = glen_H_data['Gyrometer'].rolling(window=window,center=False).mean()
glen_H_data['RollingMeanMicrophone'] = glen_H_data['Microphone'].rolling(window=window,center=False).mean()
glen_H_data['RollingMeanAltimeter'] = glen_H_data['Altimeter (Barometer) Pressure'].rolling(window=window,center=False).mean()
# Rolling 75th percentile
glen_H_data['Rolling75thAcceleration'] = glen_H_data['Acceleration'].rolling(window=window,center=False).quantile(quantile=0.75)
glen_H_data['Rolling75thMagnetometer'] = glen_H_data['Magnetometer'].rolling(window=window,center=False).quantile(quantile=0.75)
glen_H_data['Rolling75thGyrometer'] = glen_H_data['Gyrometer'].rolling(window=window,center=False).quantile(quantile=0.75)
glen_H_data['Rolling75thMicrophone'] = glen_H_data['Microphone'].rolling(window=window,center=False).quantile(quantile=0.75)
glen_H_data['Rolling75thAltimeter'] = glen_H_data['Altimeter (Barometer) Pressure'].rolling(window=window,center=False).quantile(quantile=0.75)
# Rolling st dev
glen_H_data['RollingSDAcceleration'] = glen_H_data['Acceleration'].rolling(window=window,center=False).std()
glen_H_data['RollingSDMagnetometer'] = glen_H_data['Magnetometer'].rolling(window=window,center=False).std()
glen_H_data['RollingSDGyrometer'] = glen_H_data['Gyrometer'].rolling(window=window,center=False).std()
glen_H_data['RollingSDMicrophone'] = glen_H_data['Microphone'].rolling(window=window,center=False).std()
glen_H_data['RollingSDAltimeter'] = glen_H_data['Altimeter (Barometer) Pressure'].rolling(window=window,center=False).std()
# Rolling var
glen_H_data['RollingVarAcceleration'] = glen_H_data['Acceleration'].rolling(window=window,center=False).var()
glen_H_data['RollingVarMagnetometer'] = glen_H_data['Magnetometer'].rolling(window=window,center=False).var()
glen_H_data['RollingVarGyrometer'] = glen_H_data['Gyrometer'].rolling(window=window,center=False).var()
glen_H_data['RollingVarMicrophone'] = glen_H_data['Microphone'].rolling(window=window,center=False).var()
glen_H_data['RollingVarAltimeter'] = glen_H_data['Altimeter (Barometer) Pressure'].rolling(window=window,center=False).var()

In [12]:
# Rolling Means
test['RollingMeanAcceleration'] = test['Acceleration'].rolling(window=window,center=False).mean()
test['RollingMeanMagnetometer'] = test['Magnetometer'].rolling(window=window,center=False).mean()
test['RollingMeanGyrometer'] = test['Gyrometer'].rolling(window=window,center=False).mean()
test['RollingMeanMicrophone'] = test['Microphone'].rolling(window=window,center=False).mean()
test['RollingMeanAltimeter'] = test['Altimeter (Barometer) Pressure'].rolling(window=window,center=False).mean()
# Rolling 75th percentile
test['Rolling75thAcceleration'] = test['Acceleration'].rolling(window=window,center=False).quantile(quantile=0.75)
test['Rolling75thMagnetometer'] = test['Magnetometer'].rolling(window=window,center=False).quantile(quantile=0.75)
test['Rolling75thGyrometer'] = test['Gyrometer'].rolling(window=window,center=False).quantile(quantile=0.75)
test['Rolling75thMicrophone'] = test['Microphone'].rolling(window=window,center=False).quantile(quantile=0.75)
test['Rolling75thAltimeter'] = test['Altimeter (Barometer) Pressure'].rolling(window=window,center=False).quantile(quantile=0.75)
# Rolling st dev
test['RollingSDAcceleration'] = test['Acceleration'].rolling(window=window,center=False).std()
test['RollingSDMagnetometer'] = test['Magnetometer'].rolling(window=window,center=False).std()
test['RollingSDGyrometer'] = test['Gyrometer'].rolling(window=window,center=False).std()
test['RollingSDMicrophone'] = test['Microphone'].rolling(window=window,center=False).std()
test['RollingSDAltimeter'] = test['Altimeter (Barometer) Pressure'].rolling(window=window,center=False).std()
# Rolling var
test['RollingVarAcceleration'] = test['Acceleration'].rolling(window=window,center=False).var()
test['RollingVarMagnetometer'] = test['Magnetometer'].rolling(window=window,center=False).var()
test['RollingVarGyrometer'] = test['Gyrometer'].rolling(window=window,center=False).var()
test['RollingVarMicrophone'] = test['Microphone'].rolling(window=window,center=False).var()
test['RollingVarAltimeter'] = test['Altimeter (Barometer) Pressure'].rolling(window=window,center=False).var()

In [45]:
test = test.dropna()
test_re = test.iloc[:,range(5) + range(6,26) + [5]]

test_X = test_re.iloc[:, [0, 1, 3, 4, 5, 6, 8, 9, 15, 16, 18, 19]]
test_Y = test_re.iloc[:, -1]
test_X

,Acceleration,Magnetometer,Microphone,Altimeter (Barometer) Pressure,RollingMeanAcceleration,RollingMeanMagnetometer,RollingMeanMicrophone,RollingMeanAltimeter,RollingSDAcceleration,RollingSDMagnetometer,RollingSDMicrophone,RollingSDAltimeter
2017-03-24 06:34:55,0.051671,504.654115,-25.94190,102.658073,0.106162,480.165175,-12.012938,102.501797,0.109040,30.029223,8.432274,0.059558
2017-03-24 06:35:00,0.003914,505.715961,-31.88260,102.656647,0.104774,480.950295,-12.583763,102.506096,0.109905,30.176402,8.791664,0.062800
2017-03-24 06:35:04,0.109380,505.715961,-31.88260,102.655602,0.106455,481.735415,-13.154589,102.510378,0.109307,30.302116,9.100465,0.065545
2017-03-24 06:35:09,0.193791,506.980820,-1.21374,102.654602,0.109493,482.612385,-12.852439,102.514659,0.109586,30.387127,9.242864,0.067842
2017-03-24 06:35:13,0.024156,505.866909,-10.44600,102.653542,0.107074,483.726102,-13.037089,102.518857,0.110118,30.194087,9.096667,0.069823
2017-03-24 06:35:18,0.029395,504.850147,-22.92370,102.653702,0.099427,485.005104,-13.297791,102.522845,0.101461,29.693269,9.190891,0.071729
2017-03-24 06:35:22,0.043373,506.574686,-13.91550,102.651573,0.096623,486.188878,-13.363433,102.526646,0.101025,29.340647,9.183191,0.073428
2017-03-24 06:35:27,0.027632,506.303357,-15.75700,102.653725,0.087827,488.203518,-13.609714,102.530357,0.086130,27.062013,9.076207,0.075083
2017-03-24 06:35:31,0.049980,505.534781,-18.96300,102.659157,0.082696,486.879720,-13.836317,102.534134,0.080276,24.378914,9.065371,0.076732
2017-03-24 06:35:36,0.044747,504.463659,-23.06080,102.663658,0.082240,487.661798,-13.755657,102.538014,0.080429,24.301568,8.962361,0.078295


In [46]:
glen_H_data = glen_H_data.dropna()
glen_H_data_re = glen_H_data.iloc[:,range(5) + range(6,26) + [5]]

In [47]:
X = glen_H_data_re.values[:, [0, 1, 3, 4, 5, 6, 8, 9, 15, 16, 18, 19] ]
Y = glen_H_data_re.values[:,-1]

In [48]:
RF_Class = RandomForestClassifier(n_estimators=800, n_jobs=3)

In [49]:
RF_Class.fit(X, Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=800, n_jobs=3, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [50]:
test_guess = RF_Class.predict(test_X)

In [51]:
RF_Class.score(test_X, test_Y)

0.38284250960307298

In [52]:
RF_Class.feature_importances_

array([ 0.01520157,  0.04711334,  0.02008836,  0.20414471,  0.07567609,
        0.09271063,  0.05822256,  0.22034091,  0.08798655,  0.0560968 ,
        0.04524001,  0.07717849])

In [53]:
confusion_matrix(test_Y, test_guess)

array([[  0,   0,   0,  40,  35,  46],
       [  0,   0,   0,   7,   0,   1],
       [  0,   0,   0,   0,   0,  37],
       [  0,   0,   0, 158,   0, 139],
       [  0,   0,   0,  16,   0,  68],
       [  0,   0,   0,  93,   0, 141]])

### Support Vector Machine

In [54]:
from sklearn import svm

In [55]:
#SVMs work better with scaled data, so...
from sklearn import preprocessing
X_scaled = preprocessing.scale(X)
test_X_scaled = preprocessing.scale(test_X)

In [56]:
glensvm = svm.SVC()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

param_dist = {'C': stats.expon(scale=1000), 'gamma': scipy.stats.expon(scale=1),
  'kernel': ['rbf', 'linear'], 'class_weight':['balanced', None]}

gRandSVMcv = RandomizedSearchCV(glensvm, param_dist, n_jobs=3, verbose=True)

In [ ]:
gRandSVMcv.fit(X_scaled, Y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=3)]: Done  30 out of  30 | elapsed:  6.4min finished


In [ ]:
gRandSVMcv.best_params_

In [70]:
gRandSVMcv.score(test_X_scaled, test_Y)

0.28169014084507044

In [71]:
glensvm.fit(X_scaled, Y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [72]:
glensvm.score(test_X_scaled, test_Y)

0.26248399487836105

In [59]:
test_svm = glensvm.predict(test_X_scaled)
confusion_matrix(test_Y, test_guess)

array([[  0,   0,   0,  40,  35,  46],
       [  0,   0,   0,   7,   0,   1],
       [  0,   0,   0,   0,   0,  37],
       [  0,   0,   0, 158,   0, 139],
       [  0,   0,   0,  16,   0,  68],
       [  0,   0,   0,  93,   0, 141]])